https://github.com/PacktPublishing/Python-Social-Media-Analytics/blob/5886b29587054526920e40b54ea630de5ed9d365/Chapter03/Chapter3.ipynb


https://developers.facebook.com/docs/graph-api/explorer/

https://developers.facebook.com/docs/places/search

https://developers.facebook.com/docs/graph-api/reference/page/ratings/

https://developers.facebook.com/tools/explorer/

In [ ]:
#!pip install facepy
#!pip install pony

In [1]:
import requests
import json
import time
import db_reacao
from datetime import datetime
from facepy import GraphAPI
from pony import orm

In [2]:
# Initialize the Graph API with a valid access token (optional,
# but will allow you to do all sorts of fun stuff).
graph = GraphAPI("EAACEdEose0cBAM6nekgxQ826DnyBO7IFktK90ZCe8wHyH210j0ZALrRivlzmSUZAn7ZBLtjlUY2Mx2WclOQRr9xyT4mQZCY16dBk1zEhjgdFxWTbp6ZBvZBhi5o1nZCpIVBd237ZBFbJO63TaAxcZAxPU2OWTpBQiPK3eYAAUr6euHj02RXUOV3pzK2IHWBHi6QeEZD")

In [3]:
# Get my latest posts
data_post = graph.get('v2.12/745618718900423/feed?')

In [11]:
post={}
comm={}
reac={}
reac_sentimento = {"SAD":-1,"ANGRY":-1,"HAHA":1,"LOVE":1,"WOW":1,"LIKE":1}
query_time_post = datetime.strptime("2015-01-01", '%Y-%m-%d')
query_time_comm = datetime.strptime("2018-03-01", '%Y-%m-%d')

In [ ]:
for p in range(0,len(data_post["data"])):
    post = data_post["data"][p]
    created_time_post = datetime.strptime(post.get("created_time").split("T")[0], '%Y-%m-%d')
    if created_time_post>=query_time_post:
        data_comm = graph.get('v2.12/'+ post.get("id") +'?fields=comments,reactions')
        if data_comm.get("comments")!=None:
            for c in range(0,len(data_comm["comments"]["data"])):
                comm = data_comm["comments"]["data"][c]
                created_time_comm = datetime.strptime(comm.get("created_time").split("T")[0], '%Y-%m-%d')
                if created_time_comm>=query_time_comm:
                    try:
                        db_reacao.add_reacao_face(1,2,comm["message"], 0,int(time.mktime(created_time_comm.timetuple())),str(comm["id"]))
                    except Exception as e:
                        print("erro comentario")
                        print(e)  
        if data_comm.get("reactions")!=None:
            for c in range(0,len(data_comm["reactions"]["data"])):
                reac = data_comm["reactions"]["data"][c]
                try:            
                    db_reacao.add_reacao_face(1,3,reac["type"], reac_sentimento.get(reac["type"],0),int(time.mktime(datetime.now().timetuple())),str(post["id"])+"_"+str(reac["id"]))
                except Exception as e:
                    print("erro reação")
                    print(reac["type"])
                    print(e)    